# 🧪 Tokenizer Logic Experimentation

이 노트북은 `src/utils/text.py`의 핵심 로직을 **직접 수정하고 테스트**하기 위해 코드를 가져온 버전입니다.
여기서 함수를 수정하고 바로 아래 셀에서 결과를 확인해보세요!

In [6]:
import os
import json
import ssl
from kiwipiepy import Kiwi

# SSL Context Fix for Kiwi Model Download
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# ==========================================
# 🛠️ Core Logic (from src/utils/text.py)
# ==========================================

# Singleton Instances
_kiwi = None
_stopwords = set()
_synonyms = {}

def get_kiwi():
    global _kiwi
    if _kiwi is None:
        _kiwi = Kiwi()
        # Load User Dictionary (Paths adapted for notebook location)
        possible_paths = [
            "../../data/v3/domain_dictionary.txt",  # From src/notebooks
            "data/v3/domain_dictionary.txt",        # From root
            "../data/v3/domain_dictionary.txt",     # From src
            "/Users/leemdo/Workspaces/SKN22-3rd-3Team/data/v3/domain_dictionary.txt" # Absolute
        ]
        for path in possible_paths:
            if os.path.exists(path):
                print(f"📚 Loading User Dictionary from: {path}")
                _kiwi.load_user_dictionary(path)
                break
    return _kiwi

def load_resources():
    """Loads external resources (stopwords, synonyms) if not already loaded."""
    global _stopwords, _synonyms
    
    if not _stopwords:
        paths = [
            "../../src/core/stopwords.txt",
            "src/core/stopwords.txt",
            "/Users/leemdo/Workspaces/SKN22-3rd-3Team/src/core/stopwords.txt"
        ]
        for path in paths:
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    _stopwords = set(line.strip() for line in f if line.strip())
                print(f"🛑 Loaded {len(_stopwords)} stopwords from {path}")
                break
    
    if not _synonyms:
        paths = [
            "../../src/core/synonyms.json",
            "src/core/synonyms.json",
            "/Users/leemdo/Workspaces/SKN22-3rd-3Team/src/core/synonyms.json"
        ]
        for path in paths:
            if os.path.exists(path):
                with open(path, "r", encoding="utf-8") as f:
                    grouped_data = json.load(f)
                    
                # Flatten the structure
                for standard_name, aliases in grouped_data.items():
                    if isinstance(aliases, list):
                        for alias in aliases:
                            _synonyms[alias] = standard_name
                    elif isinstance(aliases, str):
                        _synonyms[standard_name] = aliases
                        
                print(f"🔄 Loaded {len(_synonyms)} flattened synonym mappings from {path}")
                break

def tokenize_korean(text: str) -> str:
    """
    Tokenizes Korean text using Kiwi (with Domain Dictionary).
    Extracts Nouns (NN*), Verbs (VV), Adjectives (VA), Roots (XR).
    Removes Stopwords.
    Applies Synonyms.
    """
    if not text:
        return ""
        
    kiwi = get_kiwi()
    load_resources()
    
    tokens = kiwi.tokenize(text)
    
    selected_tokens = []
    for t in tokens:
        # Filter tags: Noun, Verb, Adjective, Root
        if t.tag.startswith(('N', 'V', 'XR')):
            # Filter stopwords
            if t.form not in _stopwords:
                # Synonym Replacement
                token_form = t.form
                if token_form in _synonyms:
                    token_form = _synonyms[token_form]
                    
                selected_tokens.append(token_form)
    
    return " ".join(selected_tokens)

print("✅ Tokenizer Code Loaded!")

✅ Tokenizer Code Loaded!


### 🧪 Test Cases
아래에서 토크나이징 결과를 확인하세요.

In [7]:
# 1. Breed Names & Synonyms
print(f"'러블 키우고 싶어' -> {tokenize_korean('러블 키우고 싶어')}")
print(f"'아메숏 성격' -> {tokenize_korean('아메숏 성격')}")

# 2. Slang & Jargon
print(f"'집사가 맛동산 캔다' -> {tokenize_korean('집사가 맛동산 캔다')}")

# 3. Stopwords
print(f"'고양이가 밥을 먹었다' -> {tokenize_korean('고양이가 밥을 먹었다')}")

Quantization is not supported for ArchType::neon. Fall back to non-quantized model.


📚 Loading User Dictionary from: ../../data/v3/domain_dictionary.txt
🛑 Loaded 28 stopwords from ../../src/core/stopwords.txt
🔄 Loaded 46 flattened synonym mappings from ../../src/core/synonyms.json
'러블 키우고 싶어' -> 러시안 블루 키우
'아메숏 성격' -> 아메리칸 쇼트헤어 성격
'집사가 맛동산 캔다' -> 집사 맛동산 캐
'고양이가 밥을 먹었다' -> 고양이 밥 먹
